In [11]:
#binding to kubernetes infrastructure
import pykube, json, yaml, ast, time
kubeapi = pykube.HTTPClient(pykube.KubeConfig.from_file("kubeconfs/k3s.yaml"))

In [4]:
#importing base container specs
with open('kubeconfs/neo4j_vis.yaml', "r") as file:
    neo4j_vis_specs = list(yaml.safe_load_all(file))
file.close()

In [6]:
#matching namespace of Neo4j
%store -r namespace_name
for i in range(0,len(neo4j_vis_specs)):
    neo4j_vis_specs[i]['metadata']['namespace'] = namespace_name

In [9]:
#creating deploy
if not (pykube.Deployment(kubeapi,neo4j_vis_specs[0]).exists()): pykube.Deployment(kubeapi,neo4j_vis_specs[0]).create()
if not (pykube.Service(kubeapi,neo4j_vis_specs[1]).exists()): pykube.Service(kubeapi,neo4j_vis_specs[1]).create()

In [16]:
#getting neo4j pod object
time.sleep(5)
for pod in pykube.Pod.objects(kubeapi).filter(namespace=namespace_name):
    if pod.metadata['name'].find('neovis') != -1:
        neo4j_vis_pod = pod

In [18]:
#geting pod IP if pod is ready
while not (neo4j_vis_pod.ready):
    time.sleep(5)
    neo4j_vis_pod.reload() #reload pod object (it changes while pod gets ready)
neo4j_vis_pod_ip = neo4j_vis_pod.obj['status']['podIP']

In [20]:
%store neo4j_vis_pod_ip

Stored 'neo4j_vis_pod_ip' (str)
